<a href="https://colab.research.google.com/github/shinobu357/TugasMLRaisya/blob/main/UAS/Chapter_2_Computer_Vision_Raisya_Athaya_Kamilah_101032380253.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit 2 - Convolutional Neural Networks

Unit ini membahas arsitektur Convolutional Neural Networks, yang merupakan fondasi dari banyak aplikasi computer vision modern. CNN (Convolutional Neural Network) adalah jenis jaringan saraf tiruan yang terinspirasi oleh cara otak manusia memproses informasi visual. CNN dirancang untuk memproses data dalam bentuk grid, seperti citra, dan mampu mengenali pola visual yang kompleks melalui lapisan konvolusi dan pooling.

# Arsitektur CNN dasar (Sequential)

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)), #
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()



Convoluliton untuk mengenali pola atau fitur penting dengan mengestraksi fitur pada gambar.
Pooling untuk mengurangi fitur/dimensi dengan mempertahankan informasi penting pada pola atau fitur yang telah diidentifikasi selama proses konvolusi.

Flatten berfungsi mengubah hasil konvolusi menjadi vektor datar

Fungsi Relu untuk mengubah nilai negatif menjadi nol dan membiarkan nilai positif tetap tidak berubah. ReLU memperkenalkan non-linearitas dalam model sambil menjaga komputasi efisien.

Dropout untuk Mencegah overfitting dengan menonaktifkan 50% neuron(0.5)

Dense yaitu Lapisan fully connected untuk klasifikasi dengan aktivasi softmax untuk probabilitas kelas.

# VGG19

VGG19 adalah arsitektur jaringan saraf konvolusional (CNN) yang terdiri dari 19 lapisan, termasuk 16 lapisan konvolusi dan 3 lapisan fully connected. Model ini dirancang oleh Visual Geometry Group (VGG) dari University of Oxford dan terkenal karena kesederhanaannya serta kinerjanya yang baik dalam berbagai tugas pengenalan citra. VGG19 menggunakan ukuran kernel yang konsisten (3x3) dan memiliki banyak lapisan untuk menangkap fitur gambar secara mendalam

 VGG Network Architechture

    Inputs are 224x224 images.
    Convolution kernel shape is (3,3) and max pooling window shape is (2,2).
    Number of channels for each convolutional layer 64 -> 128 -> 256 -> 512 -> 512.
    VGG16 has 16 hidden layers (13 convolutional layers and 3 fully connected layers).
    VGG19 has 19 hidden layers (16 convolutional layers and 3 fully connected layers).

In [ ]:
import torch.nn as nn


class VGG19(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG19, self).__init__()

        # Feature extraction layers: Convolutional and pooling layers
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(
                3, 64, kernel_size=3, padding=1
            ),  # 3 input channels, 64 output channels, 3x3 kernel, 1 padding
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=2, stride=2
            ),  # Max pooling with 2x2 kernel and stride 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Fully connected layers for classification
        self.classifier = nn.Sequential(
            nn.Linear(
                512 * 7 * 7, 4096
            ),  # 512 channels, 7x7 spatial dimensions after max pooling
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout layer with 0.5 dropout probability
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes),  # Output layer with 'num_classes' output units
        )

    def forward(self, x):
        x = self.feature_extractor(x)  # Pass input through the feature extractor layers
        x = x.view(x.size(0), -1)  # Flatten the output for the fully connected layers
        x = self.classifier(x)  # Pass flattened output through the classifier layers
        return x

# GoogleNet

GoogLeNet adalah arsitektur jaringan saraf konvolusional (CNN) yang diperkenalkan oleh Google pada tahun 2014. Model ini dikenal dengan penggunaan modul Inception yang memungkinkan pemilihan berbagai ukuran filter konvolusi dalam satu blok, sehingga meningkatkan efisiensi komputasi dan akurasi.

Fitur Utama GoogLeNet:

Modul Inception: Menggabungkan berbagai ukuran filter konvolusi dalam satu blok untuk menangkap fitur pada berbagai skala.
   
Pengurangan Parameter: Menggunakan teknik seperti 1x1 konvolusi untuk mengurangi jumlah parameter, sehingga meningkatkan efisiensi komputasi.
    

Auxiliary Classifiers: Menambahkan classifier tambahan pada lapisan tertentu untuk membantu pelatihan dan mengurangi overfitting

In [ ]:
import torch
import torch.nn as nn


class BaseConv2d(nn.Module):  # Menyediakan blok dasar konvolusi yang menggabungkan konvolusi 2D dan aktivasi ReLU.
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BaseConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x


class InceptionModule(nn.Module):  #Implementasi dari modul Inception, yang menggabungkan beberapa operasi konvolusi dengan ukuran kernel yang berbeda dalam satu blok untuk mengekstraksi fitur dengan berbagai skala
    def __init__(self, in_channels, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_proj):
        super(InceptionModule, self).__init__()

        self.b1 = BaseConv2d(in_channels, n1x1, kernel_size=1)  # b1: Lapisan konvolusi 1x1 untuk mengubah jumlah saluran input menjadi 1x1.

        self.b2 = nn.Sequential(    # b2: Menggunakan 1x1 konvolusi untuk mereduksi dimensi, lalu 3x3 konvolusi.
            BaseConv2d(in_channels, n3x3red, kernel_size=1),
            BaseConv2d(n3x3red, n3x3, kernel_size=3, padding=1),
        )

        self.b3 = nn.Sequential(   #b3: Menggunakan 1x1 konvolusi untuk mereduksi dimensi, lalu 5x5 konvolusi.
            BaseConv2d(in_channels, n5x5red, kernel_size=1),
            BaseConv2d(n5x5red, n5x5, kernel_size=5, padding=2),
        )

        self.b4 = nn.Sequential(  #b4: Lapisan pooling (MaxPooling), kemudian diikuti oleh 1x1 konvolusi.
            nn.MaxPool2d(3, stride=1, padding=1),
            BaseConv2d(in_channels, pool_proj, kernel_size=1),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1, y2, y3, y4], 1)


class AuxiliaryClassifier(nn.Module):  #  Menambahkan classifier pembantu untuk membantu pelatihan dan mengurangi overfitting
    def __init__(self, in_channels, num_classes, dropout=0.7):
        super(AuxiliaryClassifier, self).__init__()
        self.pool = nn.AvgPool2d(5, stride=3)
        self.conv = BaseConv2d(in_channels, 128, kernel_size=1)
        self.relu = nn.ReLU(True)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.pool(x)
        x = self.conv(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


class GoogLeNet(nn.Module): # implementasi arsitektur GoogLeNet secara keseluruhan, yang menggabungkan beberapa modul Inception dan lapisan lainnya.
    def __init__(self, use_aux=True):
        super(GoogLeNet, self).__init__()

        self.use_aux = use_aux
        ## block 1 (Menggunakan lapisan konvolusi 7x7, diikuti oleh max pooling dan Local Response Normalization (LRN) untuk stabilitas numerik.)
        self.conv1 = BaseConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.lrn1 = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, padding=1)

        ## block 2 (Block 2: Konvolusi 1x1 dan 3x3, diikuti dengan LRN dan max pooling)
        self.conv2 = BaseConv2d(64, 64, kernel_size=1)
        self.conv3 = BaseConv2d(64, 192, kernel_size=3, padding=1)
        self.lrn2 = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, padding=1)

        ## block 3 (Menggunakan beberapa modul Inception yang telah didefinisikan sebelumnya.)
        self.inception3a = InceptionModule(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, padding=1)

        ## block 4
        self.inception4a = InceptionModule(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionModule(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionModule(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionModule(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(3, stride=2, padding=1)

        ## block 5 (Dua modul Inception yang lebih besar)
        self.inception5a = InceptionModule(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionModule(832, 384, 192, 384, 48, 128, 128)

        ## auxiliary classifier (Jika use_aux diset ke True, dua auxiliary classifier ditambahkan setelah beberapa blok)
        if self.use_aux:
            self.aux1 = AuxiliaryClassifier(512, 1000)
            self.aux2 = AuxiliaryClassifier(528, 1000)

        ## block 6 ( Global average pooling dan lapisan dropout untuk regularisasi, diikuti oleh lapisan fully connected yang menghasilkan prediksi kelas)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        ## block 1
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.lrn1(x)

        ## block 2
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.lrn2(x)
        x = self.maxpool2(x)

        ## block 3
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        ## block 4
        x = self.inception4a(x)
        if self.use_aux:
            aux1 = self.aux1(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        if self.use_aux:
            aux2 = self.aux2(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        ## block 5
        x = self.inception5a(x)
        x = self.inception5b(x)

        ## block 6
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        if self.use_aux:
            return x, aux1, aux2
        else:
            return x

# Mobilenet

MobileNet adalah jenis arsitektur jaringan saraf yang dirancang untuk mobile. Arsitektur ini dikembangkan oleh tim peneliti Google dan pertama kali diperkenalkan pada tahun 2017. Tujuan utama MobileNet adalah menyediakan klasifikasi gambar dan deteksi objek berperforma tinggi dan latensi rendah pada smartphone, tablet, dan perangkat lain yang memiliki keterbatasan sumber daya.

Keunggulan utama MobileNet terletak pada penggunaan konvolusi separabel kedalaman (depthwise separable convolutions), yang secara signifikan mengurangi jumlah parameter dibandingkan dengan konvolusi standar. Konvolusi separabel kedalaman terdiri dari dua operasi: konvolusi kedalaman (depthwise convolution) dan konvolusi titik (pointwise convolution). Pendekatan ini memungkinkan MobileNet untuk mencapai keseimbangan antara akurasi dan efisiensi komputasi, menjadikannya ideal untuk aplikasi pada perangkat dengan sumber daya terbatas.


Deptwise: Operasi konvolusi dilakukan secara terpisah untuk setiap channel input (grouped convolution). Hal ini mengurangi jumlah operasi komputasi dibandingkan dengan konvolusi standar

Pointwise: Konvolusi 1x1 untuk menggabungkan informasi antar channel output dari depthwise convolution.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        # Depthwise convolution: memproses setiap channel secara independen
        self.depthwise = nn.Conv2d(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            groups=in_channels,  # "groups" sama dengan jumlah channel input
        )
        # Pointwise convolution: menggabungkan channel dengan kernel 1x1
        self.pointwise = nn.Conv2d(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0
        )

    def forward(self, x):
        x = self.depthwise(x)  # Operasi depthwise convolution
        x = self.pointwise(x)  # Operasi pointwise convolution
        return x

class MobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        # Layer awal: Konvolusi biasa
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)

        # MobileNet body: Kumpulan depthwise separable convolution
        self.dw_conv2 = DepthwiseSeparableConv(32, 64, 1)
        self.dw_conv3 = DepthwiseSeparableConv(64, 128, 2)
        self.dw_conv4 = DepthwiseSeparableConv(128, 128, 1)
        self.dw_conv5 = DepthwiseSeparableConv(128, 256, 2)
        self.dw_conv6 = DepthwiseSeparableConv(256, 256, 1)
        self.dw_conv7 = DepthwiseSeparableConv(256, 512, 2)

        # Lima konvolusi separable tambahan dengan stride 1
        self.dw_conv8 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv9 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv10 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv11 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv12 = DepthwiseSeparableConv(512, 512, 1)

        self.dw_conv13 = DepthwiseSeparableConv(512, 1024, 2)
        self.dw_conv14 = DepthwiseSeparableConv(1024, 1024, 1)

        # Layer pooling dan fully connected untuk klasifikasi
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # Output ukuran (1, 1)
        self.fc = nn.Linear(1024, num_classes)  # Klasifikasi

    def forward(self, x):
        x = self.conv1(x)  # Konvolusi awal
        x = F.relu(x)  # Aktivasi ReLU

        # MobileNet body
        x = self.dw_conv2(x)
        x = F.relu(x)
        x = self.dw_conv3(x)
        x = F.relu(x)
        x = self.dw_conv4(x)
        x = F.relu(x)
        x = self.dw_conv5(x)
        x = F.relu(x)
        x = self.dw_conv6(x)
        x = F.relu(x)
        x = self.dw_conv7(x)
        x = F.relu(x)

        # Lima depthwise separable convolution tambahan
        x = self.dw_conv8(x)
        x = F.relu(x)
        x = self.dw_conv9(x)
        x = F.relu(x)
        x = self.dw_conv10(x)
        x = F.relu(x)
        x = self.dw_conv11(x)
        x = F.relu(x)
        x = self.dw_conv12(x)
        x = F.relu(x)

        x = self.dw_conv13(x)
        x = F.relu(x)
        x = self.dw_conv14(x)
        x = F.relu(x)

        # Pooling dan fully connected
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten output
        x = self.fc(x)  # Klasifikasi
        return x

# Create the model
mobilenet = MobileNet(num_classes=1000)
print(mobilenet)

MobileNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (dw_conv2): DepthwiseSeparableConv(
    (depthwise): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
    (pointwise): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (dw_conv3): DepthwiseSeparableConv(
    (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
    (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (dw_conv4): DepthwiseSeparableConv(
    (depthwise): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128)
    (pointwise): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (dw_conv5): DepthwiseSeparableConv(
    (depthwise): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
    (pointwise): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (dw_conv6): DepthwiseSeparableConv(
    (depthwise): Conv2d(256, 256, kernel_size=(3, 3)

# Transfer Learning

Transfer learning adalah teknik pembelajaran mesin di mana model yang sudah dilatih untuk suatu tugas digunakan kembali sebagai titik awal untuk model di tugas lain. Teknik ini memanfaatkan pengetahuan dari tugas sebelumnya untuk meningkatkan pembelajaran dan performa pada tugas baru yang berhubungan.

Pendekatan Transfer Learning:
Fine-Tuning: Menyesuaikan model yang telah dilatih dengan melatih ulang beberapa lapisan akhir menggunakan data dari tugas baru. Ini memungkinkan model beradaptasi dengan spesifikasi tugas yang berbeda.

Feature Extraction: Menggunakan model yang telah dilatih sebagai ekstraktor fitur, di mana representasi yang dihasilkan digunakan untuk melatih model sederhana pada tugas baru.

timm: Library Python untuk model visi komputer yang telah dilatih sebelumnya. Mendukung banyak model arsitektur jaringan saraf dalam computer vision

In [ ]:
pip install timm

Tujuan: Menggunakan MobileNetV3-Large (versi besar dari arsitektur MobileNetV3) yang telah dilatih sebelumnya (pre-trained) pada dataset ImageNet untuk melakukan inferensi pada gambar input . Transfer learning pada MobileNetV3 berarti memanfaatkan model yang sudah dilatih sebelumnya pada dataset besar seperti ImageNet untuk mempercepat dan meningkatkan akurasi model ketika diterapkan pada tugas baru.

In [ ]:
import timm
import torch

# Load a pre-trained MobileNet model
model_name = "mobilenetv3_large_100"

model = timm.create_model(model_name, pretrained=True)

# If you want to use the model for inference
model.eval()

# Forward pass with a dummy input
# Batch size 1, 3 color channels, 224x224 image
input_tensor = torch.rand(1, 3, 224, 224)

output = model(input_tensor)
print(output)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.1M [00:00<?, ?B/s]

tensor([[-2.5577e+00,  3.7155e-01,  4.6123e-01,  9.6552e-02,  8.1252e-01,
          6.6807e-02,  6.5756e-01, -3.3988e-01, -1.9668e-01, -5.2795e-01,
          1.2036e-01,  4.1430e-01,  8.9071e-01,  5.1821e-01,  5.8612e-01,
          8.1270e-01,  1.4119e+00, -2.3977e-01,  1.7008e+00,  1.1076e+00,
          3.7075e-01,  3.3031e+00,  1.9536e+00,  2.7468e+00,  8.6084e-01,
         -4.1663e-01, -1.1914e+00, -7.6253e-01, -8.2769e-01, -1.7428e+00,
         -1.0516e+00, -7.8037e-01, -1.6514e+00,  9.6664e-01,  3.0694e+00,
         -1.4948e+00, -1.2673e+00, -1.2919e+00, -2.6515e-01, -9.4674e-01,
         -2.2127e-01, -4.2171e-02,  1.6027e+00,  3.7812e-01, -6.1439e-02,
          9.1364e-02, -5.5856e-02, -5.0446e-01, -7.9315e-01, -1.6457e+00,
         -2.0898e-01,  2.5867e-02, -8.5002e-02,  2.4259e-01,  8.4431e-01,
         -1.8692e-01,  5.5112e-01, -1.2460e+00,  1.5623e+00, -4.9573e-01,
          1.4243e+00, -2.5727e-01, -6.7993e-03, -3.0962e-02, -1.5995e-01,
          2.7360e+00,  1.3497e+00,  4.

# Resnet

ResNet adalah arsitektur jaringan saraf dalam yang diperkenalkan oleh Kaiming He dkk. pada tahun 2015. Arsitektur ini dirancang untuk mengatasi masalah degradasi yang terjadi saat melatih jaringan yang sangat dalam dengan menambahkan skip connections atau residual connections yang memungkinkan aliran informasi langsung dari satu lapisan ke lapisan lainnya tanpa melalui transformasi non-linear. Hal ini memungkinkan pelatihan jaringan yang lebih dalam tanpa penurunan kinerja.

Skip connection (atau shortcut connection) adalah sebuah jalur langsung dari satu lapisan ke lapisan lain, melompati beberapa lapisan tengah. Pada umumnya, jalur ini digunakan untuk membawa informasi dari lapisan sebelumnya langsung ke lapisan yang lebih dalam. Ini bertujuan untuk menjaga agar informasi tidak hilang atau terdistorsi saat melalui lapisan-lapisan yang lebih dalam.

Residual connection adalah varian dari skip connection yang lebih khusus. Pada residual connection, lapisan tambahan digunakan untuk menghitung perbedaan (residual) antara input dan output dari lapisan tertentu

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Import library yang dibutuhkan untuk pemrosesan gambar dan klasifikasi menggunakan pre-trained model
from transformers import ResNetForImageClassification
from transformers import AutoFeatureExtractor
import torch
from datasets import load_dataset  # Library untuk mengakses dataset dari Hugging Face Datasets

# Memuat model ResNet-50 yang sudah dilatih sebelumnya (pre-trained)
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# Mengatur model ke mode evaluasi. Ini penting untuk mematikan mekanisme dropout dan batch normalization yang hanya digunakan saat pelatihan
model.eval()

# Memuat dataset gambar untuk pengujian
dataset = load_dataset("huggingface/cats-image")  # Dataset yang berisi gambar kucing untuk pengujian
image = dataset["test"]["image"][0]  # Mengambil gambar pertama dari set data 'test'

# Menggunakan feature extractor untuk mempersiapkan gambar sebelum diberi input ke model
# Feature extractor akan mengubah gambar menjadi format tensor yang sesuai dengan input model ResNet-50
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")

# Preprocess gambar yang dipilih untuk mempersiapkannya sebagai input model
inputs = feature_extractor(image, return_tensors="pt")  # Mengembalikan input dalam format tensor PyTorch

# Menggunakan model untuk melakukan prediksi. `torch.no_grad()` digunakan untuk menghindari pencatatan gradien yang tidak diperlukan selama inferensi
with torch.no_grad():
    logits = model(**inputs).logits  # Mendapatkan output logits dari model, yang berisi nilai prediksi untuk setiap kelas

# Model memberikan prediksi salah satu dari 1000 kelas ImageNet.
# `logits.argmax(-1)` mengambil indeks kelas dengan nilai tertinggi (prediksi model)
predicted_label = logits.argmax(-1).item()  # Mengambil indeks kelas dengan probabilitas tertinggi
print(model.config.id2label[predicted_label])  # Mencetak nama kelas yang diprediksi berdasarkan indeks yang didapat
